## Aumento de dados Mixup

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.callbacks.mixup import *
from fastai.vision import *


## O que é confusão?

Este módulo contém a implementação de uma técnica de aumento de dados chamado [mixup](https://arxiv.org/abs/1710.09412). É extremamente eficiente na regularização modelos em visão computacional (que usamos para obter o nosso tempo para treinar CIFAR10 a 94% em uma GPU a 6 minutos).
Como o nome tipo de sugere, os autores do artigo confusão propor treinamento do modelo em misturas das imagens do conjunto de treinamento. Por exemplo, suponha que estamos treinando em CIFAR10. Em vez de alimentar o modelo das imagens cruas, tomamos duas imagens (não necessariamente da mesma classe) e fazer uma combinação linear deles: em termos de tensores, temos:
`New_image = t * image1 + (1-t) * image2`
onde t é uma flutuação entre 0 e 1. O alvo vamos atribuir a essa nova imagem é a mesma combinação das metas originais:
`NEW_TARGET = t * target1 + (1-t) * target2`
assumindo que os alvos são um-quente codificada (que não é o caso em PyTorch normalmente). E é tão simples como isso.
! XX_ markdown_link _xx
Cão ou gato? A resposta aqui é de 70% cão e 30% do gato!
Como a imagem acima mostra, é um pouco difícil para o olho humano a fazer sentido das imagens obtidas desta forma (embora nós ver as formas de um cão e um gato). No entanto, de alguma forma, faz muito sentido para o modelo, que treina de forma mais eficiente. Uma nota lateral importante é que quando treinando com confusão, a perda final (treinamento ou validação) será maior do que quando o treinamento sem ela, mesmo quando a precisão é muito melhor: um modelo treinados como este irá fazer previsões que são um pouco menos confiantes .

## Treinamento básico

Para testar este método, primeiro criamos um [`simple_cnn`](/layers.html#simple_cnn) e treiná-lo como fizemos com [`basic_train`](/basic_train.html#basic_train) para que possamos comparar seus resultados com uma rede treinada com confusão.

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)
model = simple_cnn((3,16,16,2))
learn = Learner(data, model, metrics=[accuracy])

In [ ]:
learn.fit(8)

epoch,train_loss,valid_loss,accuracy,time
1,0.111498,0.094612,0.965653,00:02
2,0.079887,0.064684,0.975466,00:02
3,0.053950,0.042022,0.985280,00:02
4,0.043062,0.035917,0.986752,00:02
5,0.030692,0.025291,0.989205,00:02
6,0.027065,0.024845,0.987733,00:02
7,0.031135,0.020047,0.990186,00:02
8,0.025115,0.025447,0.988714,00:02


## implementação confusão na biblioteca

No artigo original, os autores sugerem quatro coisas:
1. Crie dois DataLoaders separadas, e desenhar um lote de cada em cada iteração para misturá-las
2. Desenhar um valor para t na sequência de uma distribuição beta com uma alfa parâmetro (0,4 é sugerido no seu artigo)
3. Misturar os dois lotes, com o mesmo valor t
4. Use one-quentes alvos codificados
A implementação deste módulo é baseado nessas sugestões, e modificado onde os resultados experimentais sugeriram mudanças que melhoram o desempenho.

Os autores sugerem a utilização da distribuição beta com parâmetros alfa = 0,4. (Em geral, a distribuição beta tem dois parâmetros, mas neste caso eles vão ser iguais.) Por que eles sugerem isso? Bem, com os parâmetros que eles sugerem, a distribuição beta parece com isso:
! XX_ markdown_link _xx
o que significa que há uma probabilidade muito elevada de valores colhendo perto de 0 ou 1 (caso em que a imagem misturado é principalmente a partir de apenas uma categoria) e, em seguida, um pouco constante, muito menor probabilidade de escolher algo no meio (note que 0,33 é quase tão susceptíveis como 0,5, por exemplo).
Enquanto isso funciona muito bem, não é a maneira mais rápida, e esta é a primeira sugestão que ajustar. A desaceleração desnecessária com esta abordagem vem de desenho dois lotes diferentes em cada iteração, o que significa carregar duas vezes o número de imagens e, adicionalmente, aplicar quaisquer outras funções de aumento de dados para eles. Para evitar isso, nós aplicamos confusão em um lote com uma versão embaralhadas de si mesmo: desta forma, as imagens misturadas ainda são diferentes.
Usando o mesmo valor de `t` para todo o lote é outra sugestão que modificar. Em nossos experimentos, observamos que o modelo treinado mais rápido se desenhou um `t` diferente para cada imagem no lote. (Ambas as opções tem o mesmo resultado em termos de precisão, é apenas que um chegou há mais lentamente.)
Finalmente, observe que, com esta estratégia que pode criar imagens duplicadas: digamos que estamos misturando `image0` com` image1` e `image1` com` image0`, e que chamamos `t = 0.1` pela primeira mix e` t = 0.9` para o segundo. Então
`Image0 * 0,1 + shuffle0 * (1-0,1) = image0 * 0,1 + image1 * 0.9`
e
`Image1 * 0,9 + shuffle1 * (1-0,9) = image1 * 0.9 + * image0 0.1`
será o mesmo. É claro que temos que ser um pouco de azar para que isso aconteça, mas, na prática, vimos uma queda na precisão quando não remover duplicatas. Para evitar isso, o truque é substituir o vetor de `t` que empatou com:
`T = max (t, 1-t)`
A distribuição beta com os dois parâmetros iguais é simétrica, em qualquer caso, e desta forma podemos garantir que o maior coeficiente é sempre perto da primeira imagem (o lote não embaralhadas).

## Adicionando confusão à mistura

Nós agora adicionar [`MixUpCallback`](/callbacks.mixup.html#MixUpCallback) ao nosso aluno para que ele modifica a nossa entrada e alvo em conformidade. A função [`mixup`](/train.html#mixup) faz isso para nós nos bastidores, junto com alguns outros ajustes descritos abaixo:

In [ ]:
model = simple_cnn((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy]).mixup()
learner.fit(8)

epoch,train_loss,valid_loss,accuracy,time
1,0.358743,0.156058,0.961236,00:02
2,0.334059,0.124648,0.982336,00:02
3,0.321510,0.105825,0.987242,00:02
4,0.314596,0.099804,0.988714,00:02
5,0.314716,0.094472,0.989205,00:02
6,0.309679,0.095133,0.989696,00:02
7,0.314474,0.086767,0.990186,00:02
8,0.309931,0.095609,0.990186,00:02


Treinar com confusão melhora a melhor precisão. Note-se que a perda de validação é maior do que sem confusão, porque o modelo faz previsões menos confiantes: sem confusão, a maioria das previsões estão muito perto de 0. ou 1. (em termos de probabilidade), enquanto que o modelo com confusão faz previsões que são mais nuançada . Antes de usar confusão, certifique-se de saber se é mais importante para otimizar a perda inferior ou melhor precisão.

In [ ]:
show_doc(MixUpCallback)

<h2 id="MixUpCallback" class="doc_header"><code>class</code> <code>MixUpCallback</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/mixup.py#L8" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixUpCallback-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>MixUpCallback</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`alpha`**:`float`=***`0.4`***, **`stack_x`**:`bool`=***`False`***, **`stack_y`**:`bool`=***`True`***) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

<div class="collapse" id="MixUpCallback-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixUpCallback-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>MixUpCallback</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Callback that creates the mixed-up input and target.  

Criar um [`Callback`](/callback.html#Callback) para confusão sobre `learn` com um parâmetro alpha`` para a distribuição beta. `Stack_x` e` stack_y` determinar se nós empilhamos nossas entradas / alvos com o lambda vetor tirada ou fazer a combinação linear. (Em geral, empilhar as entradas ou saídas quando correspondem a categorias ou classes e fazer a combinação linear de outra forma.)

### métodos de retorno de chamada

Você não chama esses mesmo - eles são chamados pelo sistema [`Callback`](/callback.html#Callback) das fastai automaticamente para ativar a funcionalidade da classe.

In [ ]:
show_doc(MixUpCallback.on_batch_begin)

<h4 id="MixUpCallback.on_batch_begin" class="doc_header"><code>on_batch_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/mixup.py#L17" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixUpCallback-on_batch_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_batch_begin</code>(**`last_input`**, **`last_target`**, **`train`**, **\*\*`kwargs`**)

<div class="collapse" id="MixUpCallback-on_batch_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixUpCallback-on_batch_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_batch_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Applies mixup to `last_input` and `last_target` if `train`.  

Chama a um vector de lambda após uma distribuição beta com `self.alpha` e opera a confusão no` e `last_input` last_target` de acordo com a` `self.stack_x` e self.stack_y`.

## Lidando com a perda

Muitas vezes temos de modificar a perda de modo que seja compatível com a confusão. PyTorch era muito cuidadoso para evitar alvos de codificação one-quentes, quando possível, de modo que parece um pouco de arrastar para desfazer isso. Felizmente para nós, se a perda é um [cross-entropy](https://pytorch.org/docs/stable/nn.html#torch.nn.functional.cross_entropy) clássico, temos
`Perda (saída, NEW_TARGET) = t * perda (saída, target1) + (1-t) * perda (saída, target2)`
Então, nós não one-quente codificar qualquer coisa e, em vez apenas calcular essas duas perdas e encontrar a combinação linear.
A classe a seguir é usado para adaptar a perda de confusão. Note que a função [`mixup`](/train.html#mixup) vai usá-lo para mudar o `Learner.loss_func` se necessário.

In [ ]:
show_doc(MixUpLoss, title_level=3)

<h3 id="MixUpLoss" class="doc_header"><code>class</code> <code>MixUpLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/mixup.py#L42" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixUpLoss-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>MixUpLoss</code>(**`crit`**, **`reduction`**=***`'mean'`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="MixUpLoss-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixUpLoss-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>MixUpLoss</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Adapt the loss function `crit` to go with mixup.  

## Indocumentados Métodos - Métodos movidos abaixo desta linha irá intencionalmente ser escondido

In [ ]:
show_doc(MixUpLoss.forward)

<h4 id="MixUpLoss.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/mixup.py#L56" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixUpLoss-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`output`**, **`target`**)

<div class="collapse" id="MixUpLoss-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixUpLoss-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 